In [38]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
import glob

In [39]:
folder_path = "../../Data/"

file_list = glob.glob(os.path.join(folder_path, "*.parquet"))

df = [pd.read_parquet(file) for file in file_list]
df = pd.concat(df, ignore_index=True)

In [40]:
df, _ = train_test_split(df, train_size=20000, stratify=df['time_to_stop_activity'], random_state=11)

In [41]:
df = df.drop(
    columns = [
        "user", 
        'project', 
        'current_month', 
        'turnover_num',
        'turnover',
        'betweenness_centrality_12_intercept',
        'betweenness_centrality_12_slope', 
        'betweenness_centrality_12_std_dev',
        'betweenness_centrality_3_intercept', 
        'betweenness_centrality_3_slope',
        'betweenness_centrality_3_std_dev',
        'betweenness_centrality_6_intercept', 
        'betweenness_centrality_6_slope',
        'betweenness_centrality_6_std_dev',
        'betweenness_centrality_9_intercept', 
        'betweenness_centrality_9_slope',
        'betweenness_centrality_9_std_dev'
    ], axis = 1
)

In [42]:
df = df.astype(np.float64)
df = df.replace([np.inf], np.nan)

for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

df = df.replace([-np.inf], np.nan)

for column in df.columns:
    max_value = df[column].min(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [43]:
for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [44]:
def train_evaluate_rf(n_estimators, max_depth, min_samples_split, X_train, Y_train, X_test, Y_test):
    mse = None; mae = None; r2 = None

    try:
        rf = RandomForestRegressor(
            n_estimators = n_estimators, 
            max_depth = max_depth, 
            min_samples_split = min_samples_split
        )
        rf.fit(X_train, Y_train)
        y_pred = rf.predict(X_test)
        
        mse = mean_squared_error(Y_test, y_pred)
        mae = mean_absolute_error(Y_test, y_pred)
        r2 = r2_score(Y_test, y_pred)

        print(f'RF - Estimators: {n_estimators}, Max Depth: {max_depth}, Min Samples Split: {min_samples_split} Finalized - {mse}, {mae}, {r2}')
        
        return {
            'n_estimators': n_estimators,
            'max_depth': max_depth, 
            'min_samples_split': min_samples_split, 
            'mean_squared_error': mse,
            'mean_absolute_error': mae,
            'r2_score': r2
        }
    
    except Exception as e:
        print(f'RF - Estimators: {n_estimators}, Max Depth: {max_depth}, Min Samples Split: {min_samples_split} Error')
        return None


In [45]:
x = df.drop(columns = ['time_to_stop_activity'], axis = 1).values
y = df['time_to_stop_activity'].values

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size = 0.3, 
    random_state=11
)

In [47]:
param_combinations = [
    (n_estimators, max_depth, min_samples_split) 
    for n_estimators in [50, 100, 200]
    for max_depth in [10, 20, 30]
    for min_samples_split in [2, 5, 10]
]

In [48]:
benchmark = Parallel(n_jobs=6)(
    delayed(train_evaluate_rf)(reg, dep, min_samples, X_train, Y_train, X_test, Y_test)
    for reg, dep, min_samples in param_combinations
)

In [49]:
benchmark_df = pd.DataFrame(benchmark)

In [50]:
benchmark_df.to_excel('RandomForestRegressorBenchmark.xlsx')